In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Required installations

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
%pip install torch==1.7.1+cu92 torchvision==0.8.2+cu92 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html
%pip install torchtext 
%pip install torchvision 
%pip install torchaudio  

In [ ]:
#install spacy and download the english model
%pip install -U spacy==3.4.2
!python -m spacy download en_core_web_trf

In [ ]:
#install spacy transformers
%pip install -U spacy[cuda92,transformers]

In [ ]:

%pip install cupy-cuda92

In [ ]:
!export CUDA_PATH="/usr/local/cuda-9.2"
!export LD_LIBRARY_PATH=$CUDA_PATH/lib64:$LD_LIBRARY_PATH

In [ ]:
#script in order to fix format from ubiai (it was an error every time that a string has 2 words and the second was smaller or equal than 2 chars e.g "harbour :" or "harbour at")
import pandas as pd
dftrain = pd.read_csv("drive/MyDrive/IOB_for_bert/training_IOBall.tsv", sep='\t+', header=None, skiprows=1, engine='python', encoding="utf8")
dfdev = pd.read_csv("drive/MyDrive/IOB_for_bert/development_IOBall.tsv", sep='\t+', header=None, skiprows=1, engine='python', encoding="utf8")
dfeval = pd.read_csv("drive/MyDrive/IOB_for_bert/evaluation_IOBall.tsv", sep='\t+', header=None, skiprows=1, engine='python', encoding="utf8")

def df_collapse(df):
  for i in range(len(df[0])):
      a=df[0][i].split(" ")
      if(len(a) >= 2):
        if(len(a[1])<=2):
          df[0][i]=a[0] + a[1]
      else:
        df[0][i]=df[0][i]
  return df

dftrain=df_collapse(dftrain)
dfdev=df_collapse(dfdev)
dfeval=df_collapse(dfeval)

dftrain.to_csv("drive/MyDrive/IOB_for_bert/dftrain.tsv",index=False,sep="\t",header=None)
dfdev.to_csv("drive/MyDrive/IOB_for_bert/dfdev.tsv",index=False,sep="\t",header=None)
dfeval.to_csv("drive/MyDrive/IOB_for_bert/dfeval.tsv",index=False,sep="\t",header=None)


In [ ]:
#in order to convert from IOB to JSON
!python -m spacy convert drive/MyDrive/IOB_for_bert/dftrain.tsv ./drive/MyDrive/IOB_for_bert -t json -s -n 1 -c iob   #-n 10
!python -m spacy convert drive/MyDrive/IOB_for_bert/dfdev.tsv ./drive/MyDrive/IOB_for_bert -t json -s -n 1 -c iob
!python -m spacy convert drive/MyDrive/IOB_for_bert/dfeval.tsv ./drive/MyDrive/IOB_for_bert -t json -s -n 1 -c iob

#in order to convert from json to .spacy
!python -m spacy convert drive/MyDrive/IOB_for_bert/dftrain.json ./drive/MyDrive/IOB_for_bert -t spacy
!python -m spacy convert drive/MyDrive/IOB_for_bert/dfdev.json ./drive/MyDrive/IOB_for_bert -t spacy
!python -m spacy convert drive/MyDrive/IOB_for_bert/dfeval.json ./drive/MyDrive/IOB_for_bert -t spacy

In [ ]:
#initialise the config file
!python -m spacy init fill-config drive/MyDrive/IOB_for_bert/base_config.cfg drive/MyDrive/IOB_for_bert/config_spacy.cfg

In [ ]:
#debugging the config file
!python -m spacy debug data drive/MyDrive/IOB_for_bert/config_spacy.cfg

In [ ]:
#training of the model
!python -m spacy train -g 0 drive/MyDrive/IOB_for_bert/config_spacy.cfg  --output ./drive/MyDrive/IOB_for_bert 

In [ ]:
#evaluation of the model
!python -m spacy evaluate drive/MyDrive/IOB_for_bert/model-bestfulltxt drive/MyDrive/IOB_for_bert/evaluation_IOBall.spacy --gpu-id 0

In [ ]:
import pandas as pd

#dictionaries
df=pd.read_csv("drive/MyDrive/IOB_for_bert/dictionaries/Distribution_descriptors.csv",header=None)
df1=pd.read_csv("drive/MyDrive/IOB_for_bert/dictionaries/Life_stages.csv",header=None)
df2=pd.read_csv("drive/MyDrive/IOB_for_bert/dictionaries/Body_size.csv",header=None)
df3=pd.read_csv("drive/MyDrive/IOB_for_bert/dictionaries/Sampling_devices.csv",header=None)


df_distr_descr=df.iloc[:,0]
df_life_stages=df1.iloc[:,0]
df_body_size=df2.iloc[:,0]
df_sampl_devices=df3.iloc[:,0]

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer
nlp=spacy.blank("en")
tokenizer = nlp.tokenizer

#script for adding entity ruler-dictionaries in the model's pipeline
def entity_ruler(nlp_model,model):
    #nlp_ner_corpus2.remove_pipe("entity_ruler")
    if "entity_ruler" not in nlp_model.pipe_names:
        ruler=nlp_model.add_pipe("entity_ruler", after= "ner")
    else:
        ruler=nlp_model.get_pipe("entity_ruler")

    #GIA PATTERNS
    def dict_func(df,linkdf,label):
        patterns=[]
        j=0
        for i in df:
            dict={"label": label}
            dict["pattern"]=[{"LOWER" : str.lower(i)}]
            dict["id"]=linkdf[[1]][1][j]
            patterns.append(dict)
            tokens=tokenizer(i)
            if len(tokens) == 2:
                dict={"label": label}
                dict["pattern"]=[{"LOWER" : str.lower(str(tokens[0]))}, {"IS_PUNCT": True}, {"LOWER" : str.lower(str(tokens[1]))}]
                dict["id"]=linkdf[[1]][1][j]
                patterns.append(dict)
            j=j+1
        ruler.add_patterns(patterns)

    dict_func(df_distr_descr,df,"DISTRIBUTION_DESCRIPTOR")
    dict_func(df_life_stages,df1,"LIFE_STAGE")
    dict_func(df_body_size,df2,"BODY_SIZE")
    dict_func(df_sampl_devices,df3,"SAMPLING_DEVICE")

    #in order to save the model with entity ruler
    nlp_model.to_disk(model)


nlp_ner=spacy.load("drive/MyDrive/IOB_for_bert/model-bestfulltxt")
entity_ruler(nlp_ner,"drive/MyDrive/IOB_for_bert/model-bestfulltxt_ruler")

In [ ]:
#evaluation of the model after dictionaries addition
!python -m spacy evaluate drive/MyDrive/IOB_for_bert/model-bestfulltxt_ruler drive/MyDrive/IOB_for_bert/evaluation_IOBall.spacy --gpu-id 0

In [ ]:
import spacy
from spacy import displacy

#load trained model
nlp_ner=spacy.load("drive/MyDrive/IOB_for_bert/model-bestfulltxt")
doc2=nlp_ner("Atherinids are small marine estuarine and freshwaters fishes not exceeding 150 mm SL (a soon to be described species of Craterocephalus may reach 300 mm SL), occurring predominantly in the Old World, with only Alepidomus evermanni (freshwaters of Cuba) and two marine species, Atherinomorus stipes and Hypoatherina harringtonensis predominantly in the shore waters of the Caribbean) known from the New World. In the Southeast Pacific, the family is only known only from Tumaco, Colombia, according to Dyer. All freshwater atherinid species are derived from marine ancestors and until recently have only been known from the rivers of Australia, New Guinea and some islands in the vicinity of the latter.")

#paper=nlp_ner_corpus("Woodin (1976) highlighted the importance of adult-larval interactions in infaunal organisms, i.e. interactions between established individuals and settling and/or settled larvae, in that such interactions could generate in a shorter time and maintain more effectively definite species assemblages as compared with adult-adult interactions.")
paper2=nlp_ner("Most marine fish and invertebrate species produce free and small early-stages which are part of the plankton. These incompletely developed individuals are highly vulnerable to unsuitable conditions like starvation and environmental variability, and it was early recognized that survival during these stages often regulates recruitment and adult population size (Cowan and Shaw, 2002, Pineda et al., 2007). Recruitment theories have thus focused on the environmental modulation of larval survival, and they generally assume that while spawning occurs within relatively fixed time-frames along the year cycle, hydrographic conditions and plankton production show higher inter-annual variability.")


print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in doc2.ents])
print("\n")
print([(ent.text, ent.label_ ,ent.start_char, ent.end_char, ent.ent_id_) for ent in paper2.ents])
